In [1]:
import pandas as pd
import sys
import os

module_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ai_service import snli_label_dataset, AiClient
ai_service = AiClient()

In [4]:
train = pd.read_csv('../data/stanford-natural-language-inference-corpus/snli_test_sample_1500.csv')

In [5]:
train.head(2)

,sentence1,sentence2,gold_label
0,A kid bored in a train with brown hair and his...,A child is riding the train from New York to B...,neutral
1,A brown dog and black and white dog run along ...,Two squirrels run after acorns in the grass.,contradiction


## system promt

In [6]:
system_prompt = """
You are a high-precision classifier for the Natural Language Inference (NLI) task. Your sole goal is to determine the **logical relationship** between a pair of sentences.

Input:
1. SENTENCE 1 (Premise): The original statement.
2. SENTENCE 2 (Hypothesis): The statement under test.

Logical Relationships (Answers):
* entailment: If SENTENCE 1 **logically guarantees** the truth of SENTENCE 2.
* contradiction: If SENTENCE 1 **logically excludes** the truth of SENTENCE 2.
* neutral: If SENTENCE 1 **does not provide enough information** to determine the truth or falsity of SENTENCE 2.

Output Format:
Your answer must be **strictly one word** from the allowed list.

SENTENCE 1: {premise}
SENTENCE 2: {hypothesis}
"""

## gpt-4o-mini

In [7]:
res_df, metrics = snli_label_dataset(
    train,
    prompt_template=system_prompt,
    model_name="gpt-4o-mini",
    llm_classifier=ai_service.apenai_call
)

  0%|          | 0/1500 [00:00<?, ?it/s]

100%|██████████| 1500/1500 [15:56<00:00,  1.57it/s] 


In [8]:
display(res_df.head())
display(metrics)

,sentence1,sentence2,gold_label,pred_label,time_sec,prompt_tokens,completion_tokens,total_tokens
0,A kid bored in a train with brown hair and his...,A child is riding the train from New York to B...,neutral,neutral,1.864969,208,1,209
1,A brown dog and black and white dog run along ...,Two squirrels run after acorns in the grass.,contradiction,neutral,1.246414,205,1,206
2,A person in full astronaut suit and gear train...,A female astronaut adjusting to the feeling of...,neutral,neutral,0.591131,214,1,215
3,An old Indian man dressed in rags sleeps on th...,An old Indian man is dressed up,contradiction,contradiction,1.928916,208,3,211
4,A bunch of people are standing all together in...,A group of people are planning something.,neutral,neutral,1.367936,207,1,208


{'count': 1500,
 'avg_time': 0.6353944048881531,
 'total_cost_tokens': 311671,
 'sum_prompt_tokens': 308517,
 'sum_completion_tokens': 3154,
 'errors': 0}

In [9]:
from snli_service import get_model_summary

In [12]:
df_summary = pd.DataFrame(get_model_summary(res_df, "gpt-4o-mini")).T
display(df_summary)

c:\Users\bakru_b9pmn2k\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\bakru_b9pmn2k\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\bakru_b9pmn2k\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


,Model,NaN_Count,Total_Support,Accuracy,Support_Cleaned,Total Correct Predictions,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Correct_-,Correct_contradiction,Correct_entailment,Correct_neutral
0,gpt-4o-mini,0,1500,0.808,1500.0,1212,0.626928,0.61551,0.614659,0.819914,0.808,0.805189,0,381,377,454
